In [ ]:
pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.5 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import partial_trace, entropy, state_fidelity, DensityMatrix, Statevector
import numpy as np

# --- Circuit Setup (Same as before) ---
qc = QuantumCircuit(5, 2)
qc.h(0)
qc.cx(0, 1)
simulator = AerSimulator()
qc_transpiled = transpile(qc, simulator)
qc_transpiled.save_statevector()
result = simulator.run(qc_transpiled).result()
state = result.get_statevector()

# --- Calculate Present State (rho_P) (Using revised processing from last attempt) ---
rho_P_trace = partial_trace(state, [2, 3, 4])
rho_P_data = rho_P_trace.data
rho_P_data = 0.5 * (rho_P_data + np.conj(rho_P_data.T)) # Hermiticity
trace = np.trace(rho_P_data)
if abs(trace) > 1e-10 and abs(trace - 1.0) > 1e-9:
    rho_P_data = rho_P_data / trace
elif abs(trace) < 1e-10:
     rho_P_data = np.eye(4) / 4
rho_P = DensityMatrix(rho_P_data, dims=[2, 2])

print("--- Present State (rho_P) ---")
print(rho_P)
eigenvalues_P = np.linalg.eigvalsh(rho_P.data)
print(f"Eigenvalues(rho_P): {eigenvalues_P}") # Expect [0,0,0,1] if state remains pure
present_entropy_bits = entropy(rho_P, base=2) # Calculate entropy (expect 0)
present_entropy_bits = max(0.0, present_entropy_bits)
print(f"Present State Entropy (bits): {present_entropy_bits:.3f}")
print("-" * 20)

# --- Define Past States (rho_Hi) and Assign Proxy N_links (Same as before) ---
past_state_00 = DensityMatrix([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], dims=[2,2])
past_state_11 = DensityMatrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]], dims=[2,2])
past_state_bell = DensityMatrix([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0], [0.5, 0, 0, 0.5]], dims=[2,2])

past_states_dict = {
    "|00>": {"state": past_state_00, "N_links_proxy": 1},
    "|11>": {"state": past_state_11, "N_links_proxy": 1},
    "Bell": {"state": past_state_bell, "N_links_proxy": 10} # Higher proxy for entangled past
}

# --- Define Proxy N_links for the Present State (rho_P) - REVISED LOGIC ---
# Assign N_links_P_proxy based on the state's nature, not just entropy=0
# Check if rho_P is close to the Bell state. If so, assign high N_links proxy.
# Otherwise, assign a lower value (e.g., based on entropy if it were mixed, or default to 1).

if state_fidelity(rho_P, past_state_bell) > 0.99: # Check if it's very close to the Bell state
    N_links_P_proxy = 10.0 # Assign high proxy, same as Bell past state
    print("Present state matches Bell state, assigning N_links_P_proxy = 10.0")
elif present_entropy_bits > 0.01: # If it were significantly mixed (e.g., noise added)
     # Use entropy-based proxy if state is mixed
     N_links_P_proxy = 1 + 9 * present_entropy_bits
     N_links_P_proxy = max(1.0, N_links_P_proxy)
     print(f"Present state is mixed (S={present_entropy_bits:.3f}), assigning N_links_P_proxy = {N_links_P_proxy:.3f}")
else:
    N_links_P_proxy = 1.0 # Default to low proxy if it's pure but not Bell (or error)
    print("Present state is pure but not Bell (or error), assigning N_links_P_proxy = 1.0")

print(f"Final Present State N_links Proxy (N_links_P_proxy): {N_links_P_proxy:.3f}")
print("-" * 20)

# --- Calculate Probabilities using Paper's Formula Structure (Same calculation logic) ---
alpha = 1.0
beta = 2.0

print("--- Many-Pasts Probability Calculation (Using N_links Proxy) ---")
probabilities = {}
total_prob_weight = 0

for name, data in past_states_dict.items():
    rho_Hi = data["state"]
    N_links_Hi_proxy = data["N_links_proxy"]

    try:
        fid = state_fidelity(rho_P, rho_Hi)
        fid = np.clip(fid, 0, 1)
        D = 1.0 - fid
        delta_N_links = N_links_Hi_proxy - N_links_P_proxy
        if abs(N_links_P_proxy) > 1e-9:
            # Using formula structure P ~ exp(-alpha*D) * exp(beta * delta_N / N_P)
            entanglement_term_exponent = beta * (delta_N_links / N_links_P_proxy)
        else:
            entanglement_term_exponent = 0
        prob_weight = np.exp(-alpha * D) * np.exp(entanglement_term_exponent)
        probabilities[name] = prob_weight
        total_prob_weight += prob_weight

        print(f"Past State: {name}")
        print(f"  Fidelity(rho_P, rho_Hi): {fid:.4f}")
        print(f"  D = 1 - Fidelity: {D:.4f}")
        print(f"  N_links_Hi_proxy: {N_links_Hi_proxy}")
        print(f"  delta_N_links: {delta_N_links:.3f}")
        print(f"  Entanglement Term Exp: {entanglement_term_exponent:.3f}")
        print(f"  Prob Weight (unnormalized): {prob_weight:.4f}")

    except Exception as e:
        print(f"Calculation failed for {name}: {e}")
        probabilities[name] = 0

# --- Normalize and Print Final Probabilities ---
print("-" * 20)
print("Normalized Probabilities P(Hi|P):")
if total_prob_weight > 1e-9:
    for name, weight in probabilities.items():
        normalized_prob = weight / total_prob_weight
        print(f"  {name}: {normalized_prob:.4f}")
else:
    print("  Total probability weight is zero, cannot normalize.")
print("-" * 20)


--- Present State (rho_P) ---
DensityMatrix([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
               [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]],
              dims=(2, 2))
Eigenvalues(rho_P): [0. 0. 0. 1.]
Present State Entropy (bits): 0.000
--------------------
Present state matches Bell state, assigning N_links_P_proxy = 10.0
Final Present State N_links Proxy (N_links_P_proxy): 10.000
--------------------
--- Many-Pasts Probability Calculation (Using N_links Proxy) ---
Past State: |00>
  Fidelity(rho_P, rho_Hi): 0.5000
  D = 1 - Fidelity: 0.5000
  N_links_Hi_proxy: 1
  delta_N_links: -9.000
  Entanglement Term Exp: -1.800
  Prob Weight (unnormalized): 0.1003
Past State: |11>
  Fidelity(rho_P, rho_Hi): 0.5000
  D = 1 - Fidelity: 0.5000
  N_links_Hi_proxy: 1
  delta_N_links: -9.000
  Entanglement Term Exp: -1.800
  Prob Weight (unnormalized): 0.1003
Past State: Bell
  Fidelity(rho_P, rho_H

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
# Import noise models
from qiskit_aer.noise import NoiseModel, depolarizing_error

from qiskit.quantum_info import partial_trace, entropy, state_fidelity, DensityMatrix, Statevector
import numpy as np

# --- Noise Model Setup ---
# Example: Add simple depolarizing error to CNOT gates
error_2q = depolarizing_error(0.01, 2) # 1% depolarizing error on 2-qubit gates
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error_2q, ["cx"]) # Apply to CNOT gates only

# Use simulator with the noise model
simulator = AerSimulator(noise_model=noise_model)
# simulator = AerSimulator() # Uncomment this line to run noiseless again

# --- Circuit Setup (Same as before) ---
qc = QuantumCircuit(5, 2)
qc.h(0)
qc.cx(0, 1)
# qc.barrier() # Optional barrier for visual separation

qc_transpiled = transpile(qc, simulator)
# Use density matrix simulation method when noise is present
qc_transpiled.save_density_matrix() # Save density matrix instead of statevector

# Run simulation - may need more shots for noisy density matrix reconstruction
# result = simulator.run(qc_transpiled, shots=8192).result() # Option 1: Use shots
result = simulator.run(qc_transpiled).result() # Option 2: Use method='density_matrix' (often default)

# Retrieve the full density matrix
full_rho_data = result.data(qc_transpiled)['density_matrix']
# Convert the data (numpy array) into a DensityMatrix object
full_rho = DensityMatrix(full_rho_data)


# --- Calculate Present State (rho_P) from the full noisy density matrix ---
# Ensure the full_rho object is correctly handled
if not isinstance(full_rho, DensityMatrix):
     full_rho = DensityMatrix(full_rho)

rho_P_trace = partial_trace(full_rho, [2, 3, 4]) # Trace out qubits 2, 3, 4
rho_P_data = rho_P_trace.data

# Ensure Hermiticity
rho_P_data = 0.5 * (rho_P_data + np.conj(rho_P_data.T))
# Ensure Trace is 1
trace = np.trace(rho_P_data)
if abs(trace) > 1e-10 and abs(trace - 1.0) > 1e-9:
    rho_P_data = rho_P_data / trace
elif abs(trace) < 1e-10:
     rho_P_data = np.eye(4) / 4
# Create DensityMatrix object
rho_P = DensityMatrix(rho_P_data, dims=[2, 2])

print("--- Present State (rho_P) [Noisy Simulation] ---")
print(rho_P)
eigenvalues_P = np.linalg.eigvalsh(rho_P.data)
print(f"Eigenvalues(rho_P): {eigenvalues_P}") # Expect mixed state eigenvalues now
present_entropy_bits = entropy(rho_P, base=2)
present_entropy_bits = max(0.0, present_entropy_bits)
print(f"Present State Entropy (bits): {present_entropy_bits:.3f}") # Expect > 0 now
print("-" * 20)

# --- Define Past States and Assign Proxy N_links (Same as before) ---
past_state_00 = DensityMatrix([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], dims=[2,2])
past_state_11 = DensityMatrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]], dims=[2,2])
past_state_bell = DensityMatrix([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0], [0.5, 0, 0, 0.5]], dims=[2,2])

past_states_dict = {
    "|00>": {"state": past_state_00, "N_links_proxy": 1},
    "|11>": {"state": past_state_11, "N_links_proxy": 1},
    "Ideal Bell": {"state": past_state_bell, "N_links_proxy": 10} # Ideal Bell state as a past
}

# --- Define Proxy N_links for the Present State (rho_P) - Revised Logic ---
# Now uses entropy > 0 due to noise

# Check if the noisy state is still *closest* to Bell, despite noise
# Or base proxy purely on entropy now that it should be non-zero
if present_entropy_bits > 0.01: # If noise made it mixed
     N_links_P_proxy = 1 + 9 * present_entropy_bits # Use entropy mapping
     N_links_P_proxy = min(10.0, max(1.0, N_links_P_proxy)) # Cap between 1 and 10
     print(f"Present state is mixed (S={present_entropy_bits:.3f}), assigning N_links_P_proxy = {N_links_P_proxy:.3f}")
else: # If entropy is still near zero despite noise (unlikely)
    N_links_P_proxy = 1.0
    print("Present state entropy near zero, assigning N_links_P_proxy = 1.0")


print(f"Final Present State N_links Proxy (N_links_P_proxy): {N_links_P_proxy:.3f}")
print("-" * 20)


# --- Calculate Probabilities (Same logic) ---
alpha = 1.0
beta = 2.0
print("--- Many-Pasts Probability Calculation (Noisy Present) ---")
probabilities = {}
total_prob_weight = 0
# ... (Rest of the probability calculation loop is identical to previous code) ...

# (Paste the loop from the previous code block here)
for name, data in past_states_dict.items():
    rho_Hi = data["state"]
    N_links_Hi_proxy = data["N_links_proxy"]

    try:
        fid = state_fidelity(rho_P, rho_Hi)
        fid = np.clip(fid, 0, 1)
        D = 1.0 - fid
        delta_N_links = N_links_Hi_proxy - N_links_P_proxy
        if abs(N_links_P_proxy) > 1e-9:
            entanglement_term_exponent = beta * (delta_N_links / N_links_P_proxy)
        else:
            entanglement_term_exponent = 0
        prob_weight = np.exp(-alpha * D) * np.exp(entanglement_term_exponent)
        probabilities[name] = prob_weight
        total_prob_weight += prob_weight

        print(f"Past State: {name}")
        print(f"  Fidelity(rho_P, rho_Hi): {fid:.4f}") # Expect fidelity < 1 now even for Bell past
        print(f"  D = 1 - Fidelity: {D:.4f}") # Expect D > 0 now even for Bell past
        print(f"  N_links_Hi_proxy: {N_links_Hi_proxy}")
        print(f"  delta_N_links: {delta_N_links:.3f}")
        print(f"  Entanglement Term Exp: {entanglement_term_exponent:.3f}")
        print(f"  Prob Weight (unnormalized): {prob_weight:.4f}")

    except Exception as e:
        print(f"Calculation failed for {name}: {e}")
        probabilities[name] = 0

# --- Normalize and Print Final Probabilities ---
print("-" * 20)
print("Normalized Probabilities P(Hi|P):")
if total_prob_weight > 1e-9:
    for name, weight in probabilities.items():
        normalized_prob = weight / total_prob_weight
        print(f"  {name}: {normalized_prob:.4f}")
else:
    print("  Total probability weight is zero, cannot normalize.")
print("-" * 20)


--- Present State (rho_P) [Noisy Simulation] ---
DensityMatrix([[0.4975+0.00000000e+00j, 0.    +0.00000000e+00j,
                0.    +0.00000000e+00j, 0.495 +1.09912079e-16j],
               [0.    +0.00000000e+00j, 0.0025+0.00000000e+00j,
                0.    +0.00000000e+00j, 0.    +0.00000000e+00j],
               [0.    +0.00000000e+00j, 0.    +0.00000000e+00j,
                0.0025+0.00000000e+00j, 0.    +0.00000000e+00j],
               [0.495 -1.09912079e-16j, 0.    +0.00000000e+00j,
                0.    +0.00000000e+00j, 0.4975+0.00000000e+00j]],
              dims=(2, 2))
Eigenvalues(rho_P): [0.0025 0.0025 0.0025 0.9925]
Present State Entropy (bits): 0.076
--------------------
Present state is mixed (S=0.076), assigning N_links_P_proxy = 1.680
Final Present State N_links Proxy (N_links_P_proxy): 1.680
--------------------
--- Many-Pasts Probability Calculation (Noisy Present) ---
Past State: |00>
  Fidelity(rho_P, rho_Hi): 0.4975
  D = 1 - Fidelity: 0.5025
  N_links_Hi_pr

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.quantum_info import partial_trace, entropy, state_fidelity, DensityMatrix, Statevector
import numpy as np

# --- Simulation Parameters ---
USE_NOISE = True  # Set to True to include noise, False for ideal simulation
NOISE_LEVEL = 0.005 # Slightly lower noise level for potentially better fidelity

# --- Noise Model Setup (Optional) ---
noise_model = None
if USE_NOISE:
    error_1q = depolarizing_error(NOISE_LEVEL / 5, 1) # Smaller error for 1-qubit gates
    error_2q = depolarizing_error(NOISE_LEVEL, 2)    # Main error on 2-qubit gates
    noise_model = NoiseModel()
    # Apply errors to specific gates used in the paths
    noise_model.add_all_qubit_quantum_error(error_1q, ["h", "x", "id"])
    noise_model.add_all_qubit_quantum_error(error_2q, ["cx", "cz"])

simulator = AerSimulator(noise_model=noise_model)

# --- Define Histories (Circuits) ---
# We'll use 3 qubits: 0 (system), 1 & 2 (ancilla/environment)
# Target state P for qubit 0: |0>

# History 1 (H1): Low internal entanglement path
qc_h1 = QuantumCircuit(3, name="History_1_LowEnt")
# Simple operations, maybe just identities or single qubit gates
qc_h1.id(0) # Identity on system qubit
qc_h1.x(1)  # Flip ancilla 1
qc_h1.h(2)  # Hadamard on ancilla 2
# (Goal: Leave qubit 0 in |0>)

# History 2 (H2): High internal entanglement path
qc_h2 = QuantumCircuit(3, name="History_2_HighEnt")
# Generate temporary entanglement between ancillas, then attempt to undo
qc_h2.h(1)
qc_h2.cx(1, 2) # Entangle ancillas 1 and 2
qc_h2.barrier()
# Some operations involving the system qubit - let's just leave it alone ideally
qc_h2.id(0)
qc_h2.barrier()
# Try to disentangle ancillas (may not be perfect with noise)
qc_h2.cx(1, 2)
qc_h2.h(1)
# (Goal: Leave qubit 0 in |0>, but path involved more complex ops)

histories = {
    "H1_LowEnt": {"circuit": qc_h1},
    "H2_HighEnt": {"circuit": qc_h2}
}

# --- Define Target State P and N_links Proxies ---
# Target state P: Qubit 0 in state |0>
# Need density matrix for qubit 0 traced from a 3-qubit state |0>|psi>
# Density matrix for |0><0| on qubit 0 is [[1,0],[0,0]]
target_P_q0 = DensityMatrix([[1, 0], [0, 0]], dims=(2,))

# N_links Proxies (Heuristic assignment)
# Target state P (|0>) is simple, assign low N_links
N_links_P_proxy = 1.0
# History H1 is simple
histories["H1_LowEnt"]["N_links_proxy"] = 1.0
# History H2 involved entanglement generation/destruction
histories["H2_HighEnt"]["N_links_proxy"] = 5.0 # Assign higher proxy than H1, but maybe < Bell state

print("--- Target State P (Qubit 0 = |0>) ---")
print(target_P_q0)
print(f"N_links Proxy for P: {N_links_P_proxy}")
print("-" * 30)
print("--- Assigned History N_links Proxies ---")
print(f"H1_LowEnt: {histories['H1_LowEnt']['N_links_proxy']}")
print(f"H2_HighEnt: {histories['H2_HighEnt']['N_links_proxy']}")
print("-" * 30)


# --- Simulate Each History and Calculate Properties ---
results = {}
for name, data in histories.items():
    print(f"Simulating {name}...")
    qc = data["circuit"]
    qc.save_density_matrix(qubits=[0, 1, 2]) # Save full 3-qubit state

    # Need to handle potential transpiler issues with save instructions
    try:
        qc_transpiled = transpile(qc, simulator)
        # Check if save instruction is preserved; if not, re-add?
        # Qiskit transpiler can sometimes drop save instructions.
        # A safer way is often to simulate the statevector/density matrix directly
        # without explicit save instructions if the simulator method allows.
        # Let's try running directly without explicit save instruction if AerSimulator handles it.

        # Run simulation to get density matrix
        job = simulator.run(qc_transpiled) # Removed explicit save from circuit fed to run
        result = job.result()
        full_rho_data = result.data(qc_transpiled)['density_matrix']
        full_rho = DensityMatrix(full_rho_data)

    except Exception as e_run:
         print(f"  Simulation Run Error for {name}: {e_run}")
         results[name] = {"error": True}
         continue


    # Calculate reduced density matrix for the system qubit (qubit 0)
    rho_final_q0_trace = partial_trace(full_rho, [1, 2]) # Trace out ancillas 1, 2
    rho_final_q0_data = rho_final_q0_trace.data

    # Basic processing (Hermiticity, Trace=1)
    rho_final_q0_data = 0.5 * (rho_final_q0_data + np.conj(rho_final_q0_data.T))
    trace = np.trace(rho_final_q0_data)
    if abs(trace) > 1e-10 and abs(trace - 1.0) > 1e-9:
        rho_final_q0_data = rho_final_q0_data / trace
    elif abs(trace) < 1e-10:
        rho_final_q0_data = np.eye(2) / 2 # Default to mixed 1-qubit state

    rho_final_q0 = DensityMatrix(rho_final_q0_data, dims=(2,))

    # Calculate Fidelity and Decoherence Distance D
    try:
        fid = state_fidelity(target_P_q0, rho_final_q0) # Fidelity with target |0> state
        fid = np.clip(fid, 0, 1)
        D = 1.0 - fid
    except Exception as e_fid:
         print(f"  Fidelity Calculation Error for {name}: {e_fid}")
         D = 1.0 # Assign max distance on error

    results[name] = {
        "rho_final_q0": rho_final_q0,
        "Fidelity": fid,
        "D": D,
        "error": False
    }
    print(f"  Finished {name}: Fidelity={fid:.4f}, D={D:.4f}")


# --- Calculate Many-Pasts Probabilities ---
alpha = 1.0
beta = 2.0
probabilities = {}
total_prob_weight = 0

print("-" * 30)
print("--- Many-Pasts Probability Calculation ---")

for name, data in histories.items():
    if results[name]["error"]:
        print(f"Skipping {name} due to simulation error.")
        continue

    D = results[name]["D"]
    N_links_Hi_proxy = data["N_links_proxy"]
    delta_N_links = N_links_Hi_proxy - N_links_P_proxy # N_links(P) proxy is 1.0

    if abs(N_links_P_proxy) > 1e-9:
        entanglement_term_exponent = beta * (delta_N_links / N_links_P_proxy)
    else:
        entanglement_term_exponent = 0

    prob_weight = np.exp(-alpha * D) * np.exp(entanglement_term_exponent)
    probabilities[name] = prob_weight
    total_prob_weight += prob_weight

    print(f"History: {name}")
    print(f"  D = 1 - Fidelity: {D:.4f}")
    print(f"  N_links_Hi_proxy: {N_links_Hi_proxy:.1f}")
    print(f"  delta_N_links: {delta_N_links:.1f}")
    print(f"  Entanglement Term Exp: {entanglement_term_exponent:.3f}")
    print(f"  Prob Weight (unnormalized): {prob_weight:.4f}")

# --- Normalize and Print Final Probabilities ---
print("-" * 30)
print("Normalized History Probabilities P(Hi|P):")
if total_prob_weight > 1e-9:
    for name, weight in probabilities.items():
        normalized_prob = weight / total_prob_weight
        print(f"  {name}: {normalized_prob:.4f}")
else:
    print("  Total probability weight is zero, cannot normalize.")
print("-" * 30)


--- Target State P (Qubit 0 = |0>) ---
DensityMatrix([[1.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j]],
              dims=(2,))
N_links Proxy for P: 1.0
------------------------------
--- Assigned History N_links Proxies ---
H1_LowEnt: 1.0
H2_HighEnt: 5.0
------------------------------
Simulating H1_LowEnt...
  Finished H1_LowEnt: Fidelity=1.0000, D=0.0000
Simulating H2_HighEnt...
  Finished H2_HighEnt: Fidelity=1.0000, D=0.0000
------------------------------
--- Many-Pasts Probability Calculation ---
History: H1_LowEnt
  D = 1 - Fidelity: 0.0000
  N_links_Hi_proxy: 1.0
  delta_N_links: 0.0
  Entanglement Term Exp: 0.000
  Prob Weight (unnormalized): 1.0000
History: H2_HighEnt
  D = 1 - Fidelity: 0.0000
  N_links_Hi_proxy: 5.0
  delta_N_links: 4.0
  Entanglement Term Exp: 8.000
  Prob Weight (unnormalized): 2980.9580
------------------------------
Normalized History Probabilities P(Hi|P):
  H1_LowEnt: 0.0003
  H2_HighEnt: 0.9997
------------------------------


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.quantum_info import partial_trace, entropy, state_fidelity, DensityMatrix, Statevector
import numpy as np

# --- Simulation Parameters ---
# Noise level for the 'noisy' path CNOT gate
NOISY_PATH_ERROR_RATE = 0.05 # 5% depolarizing error on CNOT in noisy path
# Optional: Add a tiny background noise even for the 'clean' path for realism
CLEAN_PATH_ERROR_RATE = 0.001 # 0.1% depolarizing error on CNOT in clean path

# --- Noise Model Setup ---
# Noise model for the noisy path
noise_model_noisy = NoiseModel()
error_2q_noisy = depolarizing_error(NOISY_PATH_ERROR_RATE, 2)
noise_model_noisy.add_all_qubit_quantum_error(error_2q_noisy, ["cx"])

# Noise model for the clean path (optional, can be None for ideal)
noise_model_clean = NoiseModel()
error_2q_clean = depolarizing_error(CLEAN_PATH_ERROR_RATE, 2)
noise_model_clean.add_all_qubit_quantum_error(error_2q_clean, ["cx"])

# Simulators
simulator_noisy = AerSimulator(noise_model=noise_model_noisy)
simulator_clean = AerSimulator(noise_model=noise_model_clean)
# simulator_clean = AerSimulator() # Use this for an ideal clean path

# --- Define Histories (Circuits) ---
# Aim: Create Bell state |Phi+> = (|00> + |11>)/sqrt(2) on qubits 0, 1
# Use 2 qubits

# History 1 (H_clean): Standard Bell state preparation with low noise
qc_clean = QuantumCircuit(2, name="History_Clean")
qc_clean.h(0)
qc_clean.cx(0, 1)

# History 2 (H_noisy): Same circuit, but will be run with higher noise model
qc_noisy = QuantumCircuit(2, name="History_Noisy")
qc_noisy.h(0)
qc_noisy.cx(0, 1)

histories = {
    "H_Clean": {"circuit": qc_clean, "simulator": simulator_clean},
    "H_Noisy": {"circuit": qc_noisy, "simulator": simulator_noisy}
}

# --- Define Target State P and N_links Proxies ---
# Target state P: Ideal Bell state |Phi+>
rho_target_P = DensityMatrix([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0], [0.5, 0, 0, 0.5]], dims=(2,2))

# N_links Proxies (Heuristic assignment)
# Assign proxies based on the *intended* high-entanglement state/history
# We assume both paths *aimed* to create the Bell state
N_links_P_proxy = 10.0 # Proxy for the target Bell state
histories["H_Clean"]["N_links_proxy"] = 10.0 # Proxy for the clean history aiming for Bell
histories["H_Noisy"]["N_links_proxy"] = 10.0 # Proxy for the noisy history aiming for Bell

print("--- Target State P (Ideal Bell) ---")
print(rho_target_P)
print(f"N_links Proxy for P: {N_links_P_proxy}")
print("-" * 30)
print("--- Assigned History N_links Proxies ---")
print(f"H_Clean: {histories['H_Clean']['N_links_proxy']}")
print(f"H_Noisy: {histories['H_Noisy']['N_links_proxy']}")
print("-" * 30)


# --- Simulate Each History and Calculate Properties ---
results = {}
for name, data in histories.items():
    print(f"Simulating {name}...")
    qc = data["circuit"]
    sim = data["simulator"]

    # Transpile circuit for the specific simulator
    qc_transpiled = transpile(qc, sim)
    qc_transpiled.save_density_matrix()

    # Run simulation
    try:
        job = sim.run(qc_transpiled)
        result = job.result()
        rho_final_data = result.data(qc_transpiled)['density_matrix']
        rho_final = DensityMatrix(rho_final_data)

    except Exception as e_run:
         print(f"  Simulation Run Error for {name}: {e_run}")
         results[name] = {"error": True}
         continue

    # Basic processing (Hermiticity, Trace=1) - applied to the final state
    rho_final_data = 0.5 * (rho_final_data + np.conj(rho_final_data.T))
    trace = np.trace(rho_final_data)
    if abs(trace) > 1e-10 and abs(trace - 1.0) > 1e-9:
        rho_final_data = rho_final_data / trace
    elif abs(trace) < 1e-10:
        rho_final_data = np.eye(4) / 4
    rho_final = DensityMatrix(rho_final_data, dims=(2,2))


    # Calculate Fidelity and Decoherence Distance D relative to the TARGET state
    try:
        fid = state_fidelity(rho_target_P, rho_final) # Fidelity with target ideal Bell state
        fid = np.clip(fid, 0, 1)
        D = 1.0 - fid
    except Exception as e_fid:
         print(f"  Fidelity Calculation Error for {name}: {e_fid}")
         D = 1.0 # Assign max distance on error

    results[name] = {
        "rho_final": rho_final,
        "Fidelity_to_Target": fid,
        "D": D,
        "error": False
    }
    print(f"  Finished {name}: Fidelity to Target={fid:.4f}, D={D:.4f}")
    print(f"  Final State Entropy (bits): {max(0.0, entropy(rho_final, base=2)):.4f}")


# --- Calculate Many-Pasts Probabilities ---
alpha = 1.0 # As suggested by paper Appendix A.3.1
beta = 2.0  # As suggested by paper Appendix A.3.1
probabilities = {}
total_prob_weight = 0

print("-" * 30)
print("--- Many-Pasts Probability Calculation ---")

for name, data in histories.items():
    if results[name]["error"]:
        print(f"Skipping {name} due to simulation error.")
        continue

    D = results[name]["D"] # Decoherence distance from target P
    N_links_Hi_proxy = data["N_links_proxy"] # N_links assigned to this history
    # N_links_P_proxy is the proxy for the target state P
    delta_N_links = N_links_Hi_proxy - N_links_P_proxy # This will be 0 for both paths in this setup

    # Calculate entanglement term exponent
    if abs(N_links_P_proxy) > 1e-9:
        entanglement_term_exponent = beta * (delta_N_links / N_links_P_proxy)
    else:
        entanglement_term_exponent = 0
    # Note: Since delta_N_links is 0, this exponent will be 0 for both

    # Calculate probability weight
    prob_weight = np.exp(-alpha * D) * np.exp(entanglement_term_exponent)
    # Simplified: prob_weight = np.exp(-alpha * D) since entanglement exponent is 0
    probabilities[name] = prob_weight
    total_prob_weight += prob_weight

    print(f"History: {name}")
    print(f"  D = 1 - Fidelity: {D:.4f}")
    print(f"  N_links_Hi_proxy: {N_links_Hi_proxy:.1f}")
    print(f"  delta_N_links: {delta_N_links:.1f}") # Should be 0.0
    print(f"  Entanglement Term Exp: {entanglement_term_exponent:.3f}") # Should be 0.000
    print(f"  Prob Weight (unnormalized) = exp(-alpha*D): {prob_weight:.4f}")

# --- Normalize and Print Final Probabilities ---
print("-" * 30)
print("Normalized History Probabilities P(Hi|P):")
if total_prob_weight > 1e-9:
    for name, weight in probabilities.items():
        normalized_prob = weight / total_prob_weight
        print(f"  {name}: {normalized_prob:.4f}")
else:
    print("  Total probability weight is zero, cannot normalize.")
print("-" * 30)

--- Target State P (Ideal Bell) ---
DensityMatrix([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
               [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]],
              dims=(2, 2))
N_links Proxy for P: 10.0
------------------------------
--- Assigned History N_links Proxies ---
H_Clean: 10.0
H_Noisy: 10.0
------------------------------
Simulating H_Clean...
  Finished H_Clean: Fidelity to Target=0.9992, D=0.0008
  Final State Entropy (bits): 0.0101
Simulating H_Noisy...
  Finished H_Noisy: Fidelity to Target=0.9625, D=0.0375
  Final State Entropy (bits): 0.2901
------------------------------
--- Many-Pasts Probability Calculation ---
History: H_Clean
  D = 1 - Fidelity: 0.0008
  N_links_Hi_proxy: 10.0
  delta_N_links: 0.0
  Entanglement Term Exp: 0.000
  Prob Weight (unnormalized) = exp(-alpha*D): 0.9993
History: H_Noisy
  D = 1 - Fidelity: 0.0375
  N_links_Hi_proxy: 10.0
  delta_N_links: 0.0

<ipython-input-9-3bb8aaf22d81>:93: DeprecationWarning: The return type of saved density matrices has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.DensityMatrix` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  rho_final_data = 0.5 * (rho_final_data + np.conj(rho_final_data.T))
